##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object detection

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>


This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Imports and function definitions


In [2]:
#@title Imports and function definitions

# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
import requests
from six.moves.urllib.request import urlopen
from six import BytesIO
import urllib
import time

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# File system
import glob

# Async calls
import asyncio

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

The following GPU devices are available: 


## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [9]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
    #urllib.request.urlretrieve("https://www.interregemr.eu/IManager/Media/65321/1599724/EN/add_tablet/people-to-people.jpg", './img.JPG')  
    #urllib.request.urlretrieve("https://img.kyodonews.net/english/public/images/posts/710ac39eef20bca3eb59b18a4bd584b5/cropped_image_l.jpg", "./img.JPG")
    #urllib.request.urlretrieve("http://10.5.5.101:5000/static/img.JPG", "./img.JPG")

    try:
        for filename in glob.glob('*.JPG'):
            im = Image.open(filename)
            pil_image = ImageOps.fit(im, (new_width, new_height), Image.ANTIALIAS)
            pil_image_rgb = pil_image.convert("RGB")
            pil_image_rgb.save(filename, format="JPEG", quality=90)
            print("Image downloaded to %s." % filename)
            if display:
                display_image(pil_image)
    except:
        pass
    return 'img.JPG'


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i].tolist())
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

## Apply module

Load a public image from Open Images v4, save locally, and display.

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [5]:
def process(downloaded_image_path):
    module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
    #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]
    #module_handle = "faster_rcnn_nas_coco_2018_01_28"

    with tf.Graph().as_default():
      detector = hub.Module(module_handle)
      image_string_placeholder = tf.placeholder(tf.string)
      decoded_image = tf.image.decode_jpeg(image_string_placeholder)
      # Module accepts as input tensors of shape [1, height, width, 3], i.e. batch
      # of size 1 and type tf.float32.
      decoded_image_float = tf.image.convert_image_dtype(
          image=decoded_image, dtype=tf.float32)
      module_input = tf.expand_dims(decoded_image_float, 0)
      result = detector(module_input, as_dict=True)
      init_ops = [tf.global_variables_initializer(), tf.tables_initializer()]

      session = tf.Session()
      session.run(init_ops)

      # Load the downloaded and resized image and feed into the graph.
      with tf.gfile.Open(downloaded_image_path, "rb") as binfile:
        image_string = binfile.read()

      result_out, image_out = session.run(
          [result, decoded_image],
          feed_dict={image_string_placeholder: image_string})
      print("Found %d objects." % len(result_out["detection_scores"]))

    image_with_boxes = draw_boxes(
        np.array(image_out), result_out["detection_boxes"],
        result_out["detection_class_entities"], result_out["detection_scores"] > 0.2)
    unique, counts = np.unique(result_out['detection_class_entities'][result_out['detection_scores'] > 0.2], return_counts=True)
    # Just decode keys into string
    unique = [u.decode("utf-8") for u in unique]
    display_image(image_with_boxes)
    
    headers = {
    'Content-Type': 'application/json',
    }
    number_of_persons = 0
    nb_objects = dict(zip(unique, counts))
    if 'Human face' in nb_objects:
        number_of_persons = nb_objects['Human face']
    else:
        if 'Man' in nb_objects:
            number_of_persons += nb_objects['Man']
        if 'Woman' in nb_objects:
            number_of_persons += nb_objects['Woman']
    data = '{"ts":time.time(), "number_of_persons":' + str(number_of_persons) + '}'

    r = requests.post('http://localhost:9090/api/v1/A1_TEST_TOKEN/telemetry', headers=headers, data=data)
    print(r.status_code, r.reason)

In [6]:
def process_and_show():
    image_path = '../'
    #downloaded_image_path = './'

    downloaded_image_path = download_and_resize_image(image_path, 1280, 856, False)
    print('path', downloaded_image_path)
    try:
        process(downloaded_image_path)
    except:
        pass

    
    

In [7]:
def main():
    while True:
        try:
            process_and_show()
        except:
            pass
        time.sleep(10)

In [15]:
main()

path img.JPG
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
path img.JPG
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
path img.JPG
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
path img.JPG
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
path img.JPG
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
path img.JPG
INFO:tensorflow:Saver not create

KeyboardInterrupt: 

In [10]:
process_and_show()

path img.JPG
INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [22]:
tf.reset_default_graph()